In [2]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
import os, random, sys 
ROOT = "/content/drive"     # default location for the drive
 
drive.mount(ROOT, force_remount=True) 

Mounted at /content/drive


In [3]:
# set file path
path = "/content/drive/My Drive/cancer_class"
os.chdir(path)
!ls

bi_lstm.h5	 cnn_best.h5   hybrid.tflite
cancer-class.py  cnn_model.h5  test_imgs


### Download model from Kaggle kernel 

In [ ]:
!wget https://kkb-production.jupyter-proxy.kaggle.net/k/67550201/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..zgOsKN-9o2m09yMrgH278A.JqKx2oiB2J0GBC4GfVnYxTSmUdZCDCUqNvdI9HhOXT-qJ_Pakb4QwgbM1KVikSnk2upsjcy4774M1KWH4XQjUBWLJTEWQn139XMn7iFfoFjMKLCgVTMnF42AUJPKenw3f936q2EzVNSzv9lIDXnXUzlgIwl8bgf1OxpCI3VNt8PE5PLdR2v1yjywinsAzwZhYjpFTDkuX49hA5Q-u4Skkw.yLuxmoAndXOUxB7jKMtjCA/proxy/files/models/bi_lstm.h5

--2021-07-06 00:51:07--  https://kkb-production.jupyter-proxy.kaggle.net/k/67550201/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..zgOsKN-9o2m09yMrgH278A.JqKx2oiB2J0GBC4GfVnYxTSmUdZCDCUqNvdI9HhOXT-qJ_Pakb4QwgbM1KVikSnk2upsjcy4774M1KWH4XQjUBWLJTEWQn139XMn7iFfoFjMKLCgVTMnF42AUJPKenw3f936q2EzVNSzv9lIDXnXUzlgIwl8bgf1OxpCI3VNt8PE5PLdR2v1yjywinsAzwZhYjpFTDkuX49hA5Q-u4Skkw.yLuxmoAndXOUxB7jKMtjCA/proxy/files/models/bi_lstm.h5
Resolving kkb-production.jupyter-proxy.kaggle.net (kkb-production.jupyter-proxy.kaggle.net)... 35.244.180.134
Connecting to kkb-production.jupyter-proxy.kaggle.net (kkb-production.jupyter-proxy.kaggle.net)|35.244.180.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505462240 (482M) [application/octet-stream]
Saving to: ‘bi_lstm.h5’

bi_lstm.h5          100%[===================>] 482.05M  28.9MB/s    in 17s     

2021-07-06 00:51:25 (27.9 MB/s) - ‘bi_lstm.h5’ saved [505462240/505462240]



### Import libraries

In [6]:
!pip install -q efficientnet

     |████████████████████████████████| 51kB 8.0MB/s 


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import ( Dense, Flatten, Dropout, LSTM, 
                                     Bidirectional, Lambda, Reshape,
                                    GlobalAveragePooling2D ) 
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras import optimizers, preprocessing
# import efficientnet.tfkeras as efn
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

2021-07-10 21:49:06.424895: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [15]:
print(tf.__version__)

2.5.0


### Load Keras Model and Convert to TFLite

In [8]:
model = load_model('bi_lstm.h5') 

In [12]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('hybrid.tflite', 'wb') as f:
  f.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpudwgbd63/assets


INFO:tensorflow:Assets written to: /tmp/tmpudwgbd63/assets


### Model Inference

In [4]:
labels_dict = {0: 'Benign', 1: 'Malignant'}
# load and resize image
img = Image.open('/content/1035px-Melanoma.jpg')
test_image = img.resize((512,512))
test_image = preprocessing.image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="hybrid.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on image
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0]['index'])
pred_class = (pred > 0.5).astype("int32")
cancer_class = labels_dict[pred_class.item()]
result = f"Prediction: {cancer_class} "
print(result)

Prediction: Malignant 
